In [972]:
import numpy as np
import re

In [973]:
f = open("./input", "r")
file = f.read().split("\n\n")

In [975]:
grid = np.array([[x for x in l] for l in file[0].splitlines()])
moves = np.array([[x for x in l] for l in file[1].splitlines()])
moves = moves.reshape((moves.shape[0]*moves.shape[1]))

In [976]:
from operator import add, sub, mul
def tsub(a, b):
    return tuple(map(sub, a, b))
def tadd(a, b):
    return tuple(map(add, a, b))
def tmul(a, b):
    return tuple(map(mul, a, b))
def print_grid(g):
    print("\n".join(["".join(x) for x in g]), flush=True)

In [977]:
start = tuple(zip(*np.where(grid == "@")))[0]
obstacles = list(tuple(zip(*np.where(grid == "#"))))
boxes = list(tuple(zip(*np.where(grid == "O"))))

In [978]:
import bisect

OPTIONS = ["^", ">", "v", "<"]
OPTIONS_I = {"^": 0, ">": 1, "v": 2, "<": 3}
STRATEGIES = {"^": (-1, 0), ">": (0, 1), "v": (1, 0), "<": (0, -1)}

o_map = [[[None, None, None, None] for y in range(grid.shape[1])] for x in range(grid.shape[0])]
obstacle = np.where(grid == "#")
o_rows = {key: [] for key in range(len(grid))}
for i, o in enumerate(obstacle[0]):
    o_rows[o].append(obstacle[1][i])
o_cols = {key: [] for key in range(len(grid))}
for i, o in enumerate(obstacle[1]):
    o_cols[o].append(obstacle[0][i])
next_o_strategy = {
    "^": lambda curr: (o_cols[curr[1]][bisect.bisect_left(o_cols[curr[1]], curr[0]) - 1], curr[1]),
    ">": lambda curr: (curr[0], o_rows[curr[0]][bisect.bisect_right(o_rows[curr[0]], curr[1])]),
    "v": lambda curr: (o_cols[curr[1]][bisect.bisect_right(o_cols[curr[1]], curr[0])], curr[1]),
    "<": lambda curr: (curr[0], o_rows[curr[0]][bisect.bisect_left(o_rows[curr[0]], curr[1]) - 1]),
}
for xi in range(1, grid.shape[0] - 1):
    for yi in range(1, grid.shape[1] - 1):
        for oi, o in enumerate(OPTIONS):
            o_map[xi][yi][oi] = next_o_strategy[o]((xi, yi))

In [979]:
from copy import deepcopy
c = start

boxes_new = deepcopy(boxes)

for direction in moves:
    next_o = o_map[c[0]][c[1]][OPTIONS_I[direction]]
    diff = tsub(next_o, c)
    dim_stay = 0 if direction in ["<", ">"] else 1
    dim_move = abs(dim_stay - 1)
    boxes_in_line = sorted((filter(lambda b: (b[dim_stay]) == (c[dim_stay]), boxes_new)), key=lambda x: x[dim_move])
    boxes_affected = []
    next_b = c
    while True:
        next_b = tadd(next_b, STRATEGIES[direction])
        if next_b not in boxes_in_line:
            break
        boxes_affected.append(next_b)
    space = abs(diff[dim_move]) - len(boxes_affected) - 1
    if space == 0:
        continue
    # print(new_c)
    for i, box in enumerate(boxes_affected):
        bi = boxes_new.index(box)
        boxes_new[bi] = tadd(box, STRATEGIES[direction])
    c = tadd(c, STRATEGIES[direction])

In [980]:
part1 = 0
for bx, by in boxes_new:
    part1 += bx *100 + by
part1

1294459

In [981]:
g = np.array([["." for y in range(grid.shape[0]*2)] for x in range(grid.shape[0])])
for o in obstacles:
    g[tmul(o, (1, 2))] = "#"
    g[tadd(tmul(o, (1, 2)), (0,1))] = "#"
for b in boxes:
    g[tmul(b, (1, 2))] = "["
    g[tadd(tmul(b, (1, 2)), (0,1))] = "]"
g[tmul(start, (1, 2))] = "@"
s = tmul(start, (1, 2))

In [982]:
import time 
from IPython.display import clear_output

def check_expand(curr, dir):
    n = tadd(curr, STRATEGIES[dir])
    next_blocks = []
    if g[n] == "#":
        return [], True
    if g[n] == "[":
        next_blocks.append(n)
        next_blocks.append(tadd(n, (0, 1)))
    if g[n] == "]":
        next_blocks.append(n)
        next_blocks.append(tadd(n, (0, -1)))
    all_blocks = []
    for block in next_blocks:
        new_blocks, is_blocked = check_expand(block, dir)
        if is_blocked:
            return [], is_blocked
        all_blocks.append(block)
        all_blocks += new_blocks
    return all_blocks, False


c = s
for direction in moves:
    affected_boxes = []
    is_blocked = False
    if direction in ["<", ">"]:
        dim_stay = 0
        dim_move = 1
        check = c
        while True:
            check = tadd(check, STRATEGIES[direction])
            if g[check] in ["[", "]"]:
                affected_boxes.append(check)
                continue
            if g[check] == "#":
                is_blocked = True
            break
    else:  # ^ v
        affected_boxes, is_blocked = check_expand(c, direction)

    if is_blocked:
        continue
    box_values = [g[b] for b in affected_boxes]
    for box in affected_boxes:
        g[box] = "."
    for bi, box in enumerate(affected_boxes):
        g[tadd(box, STRATEGIES[direction])] = box_values[bi]

    g[c] = "."
    c = tadd(c, STRATEGIES[direction])
    g[c] = "@"

    # clear_output(wait=True)
    # print_grid(g)
    # time.sleep(0.07)

In [983]:
part2 = 0
for x in range(g.shape[0]):
    for y in range(g.shape[1]):
        if g[(x,y)] == "[":
            part2 += x *100 + y
part2

1319212